### Importing required libraries

In [23]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NSP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NSP


In [24]:
from IPython.display import clear_output
# Installing deeplake library for dataset
!pip install deeplake
clear_output(wait=False)

In [25]:
from albumentations import (Compose,
                            RandomSizedBBoxSafeCrop,
                            BboxParams,
                            HorizontalFlip,
                            Normalize)
from albumentations.pytorch import ToTensorV2
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

import deeplake
import numpy as np
import math
import sys
import time
import torchvision
import torch
import torchvision.models.detection.mask_rcnn

### Data Collection and Preprocessing
#### Dataset: https://datasets.activeloop.ai/docs/ml/datasets/coco-dataset/

In [27]:
# Getting data from DeepLake so as to reduce the memory usage
data = deeplake.load('hub://activeloop/coco-val')

# Number of categories in the COCO dataset
num_classes = len(data.categories.info.class_names)
print("Number of Classes: ", num_classes)

hub://activeloop/coco-val loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/coco-val
Number of Classes:  80


### Creating the custom dataset
Reference: 
1. https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
2. https://pytorch.org/vision/stable/_modules/torchvision/datasets/coco.html
3. https://www.section.io/engineering-education/understanding-coco-dataset/

In [28]:
from dataclasses import dataclass
# Data transformations using Albumentations
# Reference: https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

# (428, 640) | (427, 640) ==> (128, 128)
data_transformations = Compose([RandomSizedBBoxSafeCrop(width=128, height=128), HorizontalFlip(p=0.5), Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), ToTensorV2()], 
                      # Transforming bounding boxes when an image is cropped.
                      bbox_params=BboxParams(format='pascal_voc', label_fields=['class_labels', 'bbox_ids'], min_area=25, min_visibility=0.6))



# Transforming the data to feed into the model
def transform_data(data):

    # Changing the boxes to Pascal VOC format for easier computations
    # COCO: [x-top left, y-top left, width, height]  ===> Pascal VOC: [xmin-top left, ymin-top left, xmax-bottom right, ymax-bottom right]
    boxes = convertToPascalVOC(data['boxes'])

    # Getting the images from the data
    images = data['images']

    # Check if any grayscale images are present and convert them into RGB format
    if images.shape[2] == 1:
        # Simply repeating the values of a single array into 3D array: https://numpy.org/doc/stable/reference/generated/numpy.repeat.html
        images = np.repeat(images, int(3/images.shape[2]), axis = 2) 


    # Transforming the data
    transformed_data = data_transformations(image = images, # Images
                              masks = [data['masks'][:,:,i].astype(np.uint8) for i in range(data['masks'].shape[2])], # Masks
                              bboxes = boxes, # Bounding Boxes
                              bbox_ids = np.arange(boxes.shape[0]), # Bounding Box Id's
                              class_labels = data['categories'], # Categories/labels
                              )

    # Converting transformed data into tensors
    # Converting labels
    labels = torch.tensor(transformed_data['class_labels'], dtype = torch.int64)

    # Converting boxes
    boxes = torch.zeros((len(transformed_data['bboxes']), 4), dtype = torch.int64)
    for i, box in enumerate(transformed_data['bboxes']):
        boxes[i,:] = torch.tensor(np.round(box))
        

    # Converting masks
    masks = torch.zeros((len(transformed_data['bbox_ids']), transformed_data['image'].shape[1], transformed_data['image'].shape[2]), dtype = torch.int64)
    # Check if a bounding box is dropped. If dropped don't extract mask from that bounding box
    if len(transformed_data['bbox_ids'])>0:
        # the model accepts masks as a stack, since an image may contain multiple object instances
        masks = torch.tensor(np.stack([transformed_data['masks'][i] for i in transformed_data['bbox_ids']], axis = 0), dtype = torch.uint8)
    

    # Saving the annotations into a dictionary object
    target = {'masks': masks, 
              'labels': labels, 
              'boxes': boxes}

    return transformed_data['image'], target


# Helper function to convert bounding boxes from COCO to PascalVOC format
def convertToPascalVOC(boxes):
    bboxes = np.stack((boxes[:,0], boxes[:,1], 
                       boxes[:,0]+np.clip(boxes[:,2], 1, None), # Clipping bounding boxes so that negative values of width and height are removed
                       boxes[:,1]+np.clip(boxes[:,3], 1, None)), # Source: https://numpy.org/doc/stable/reference/generated/numpy.clip.html
                       axis = 1)
    return bboxes

# To convert the data from transform_data() into a list of tuples
def collate_fn(batch):
    return tuple(zip(*batch))

In [29]:
# Batch Size
batch_size = 8

# Batching the data and loading into a Data Loader
data_loader = data.pytorch(num_workers = 2, shuffle = False,
    # loading the required data only i.e images, masks, labels and bounding boxes 
    tensors = ['images', 'masks', 'categories', 'boxes'],
    transform = transform_data, 
    batch_size = batch_size,
    collate_fn = collate_fn)

/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


### Custom Model
Reference: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [30]:
# Helper function for loading the model
def semantic_segmentation_model(num_classes):
    # Mask RCNN model
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    # Extracting features for bounding boxes
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    # Extracting features for masks
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [31]:
# To move model from CPU to GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Model instance
model = semantic_segmentation_model(num_classes)
model.to(device)

# Model configurations
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Model Training

In [14]:
len_dataloader = len(data_loader)
print("Length of Data Loader: ", len_dataloader)

# Epochs
num_epochs = 5

# Model Training
for epoch in range(num_epochs):
    print(f"Epoch: {epoch}/{num_epochs}")
    model.train()
    loss = []
    i = 0
    # Load images and annotations from data loader
    for images, annotations in data_loader:

        images = list(image.to(device) for image in images)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        # Training the model on images and annotations
        loss_dict = model(images, annotations)
        
        losses = sum(loss for loss in loss_dict.values()) # Total loss from model training on each instance
        
        optimizer.zero_grad() # Setting the gradients of the optimizer to zero. Removes gradients from previous calculations
        losses.backward() # Calculating the gradients and applying backpropogation algorithm
        optimizer.step()# Updates the model parameters

        loss.append(losses.item()) # Loss from each instance of data loader
        i+=1
        print(f"Iteration: {i}/{len_dataloader}, Loss: {losses}")

    
    print("# ----------------------- Losss -------------------- #")    
    print(f"Loss {losses.item()}")
    print("# -------------- #### -------------- #")

Iteration: 1/625, Loss: 2.171905279159546
Iteration: 2/625, Loss: 66.67845916748047
Iteration: 3/625, Loss: 1.9914309978485107
Iteration: 4/625, Loss: 2.4169321060180664
Iteration: 5/625, Loss: 2.5600476264953613
Iteration: 6/625, Loss: 3.6305854320526123
Iteration: 7/625, Loss: 1.7614166736602783
Iteration: 8/625, Loss: 1.8333786725997925
Iteration: 9/625, Loss: 1.7196242809295654
Iteration: 10/625, Loss: 1.8992794752120972
Iteration: 11/625, Loss: 1.6060081720352173
Iteration: 12/625, Loss: 2.253326654434204
Iteration: 13/625, Loss: 2.385554313659668
Iteration: 14/625, Loss: 1.7894604206085205
Iteration: 15/625, Loss: 1.7853127717971802
Iteration: 16/625, Loss: 1.6081396341323853
Iteration: 17/625, Loss: 1.6640145778656006
Iteration: 18/625, Loss: 2.034580945968628
Iteration: 19/625, Loss: 1.5777643918991089
Iteration: 20/625, Loss: 1.5054911375045776
Iteration: 21/625, Loss: 1.5991514921188354
Iteration: 22/625, Loss: 1.398391842842102
Iteration: 23/625, Loss: 2.0361220836639404
Ite

/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Iteration: 1/625, Loss: 1.4747002124786377
Iteration: 2/625, Loss: 1.1560535430908203
Iteration: 3/625, Loss: 1.0975840091705322
Iteration: 4/625, Loss: 1.4416910409927368
Iteration: 5/625, Loss: 1.4677910804748535
Iteration: 6/625, Loss: 1.2467597723007202
Iteration: 7/625, Loss: 1.2191708087921143
Iteration: 8/625, Loss: 1.0177315473556519
Iteration: 9/625, Loss: 1.2575688362121582
Iteration: 10/625, Loss: 1.2656610012054443
Iteration: 11/625, Loss: 1.165569543838501
Iteration: 12/625, Loss: 1.4924553632736206
Iteration: 13/625, Loss: 1.565138578414917
Iteration: 14/625, Loss: 1.3444337844848633
Iteration: 15/625, Loss: 1.399748682975769
Iteration: 16/625, Loss: 1.2622613906860352
Iteration: 17/625, Loss: 1.4023252725601196
Iteration: 18/625, Loss: 1.5952427387237549
Iteration: 19/625, Loss: 1.3595890998840332
Iteration: 20/625, Loss: 1.3269044160842896
Iteration: 21/625, Loss: 1.4203693866729736
Iteration: 22/625, Loss: 1.359144687652588
Iteration: 23/625, Loss: 1.7200230360031128
I

/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Iteration: 1/625, Loss: 1.5008476972579956
Iteration: 2/625, Loss: 1.0597261190414429
Iteration: 3/625, Loss: 1.0615530014038086
Iteration: 4/625, Loss: 1.2752234935760498
Iteration: 5/625, Loss: 1.4546493291854858
Iteration: 6/625, Loss: 1.2298188209533691
Iteration: 7/625, Loss: 1.2235078811645508
Iteration: 8/625, Loss: 0.9814395904541016
Iteration: 9/625, Loss: 1.197959542274475
Iteration: 10/625, Loss: 1.280378818511963
Iteration: 11/625, Loss: 1.1399716138839722
Iteration: 12/625, Loss: 1.5065349340438843
Iteration: 13/625, Loss: 1.4734035730361938
Iteration: 14/625, Loss: 1.310076355934143
Iteration: 15/625, Loss: 1.4168330430984497
Iteration: 16/625, Loss: 1.1740915775299072
Iteration: 17/625, Loss: 1.4346370697021484
Iteration: 18/625, Loss: 1.4858779907226562
Iteration: 19/625, Loss: 1.4398707151412964
Iteration: 20/625, Loss: 1.2519447803497314
Iteration: 21/625, Loss: 1.5273463726043701
Iteration: 22/625, Loss: 1.266264796257019
Iteration: 23/625, Loss: 1.6718370914459229
I

/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Iteration: 1/625, Loss: 1.5116087198257446
Iteration: 2/625, Loss: 1.0720100402832031
Iteration: 3/625, Loss: 1.0651772022247314
Iteration: 4/625, Loss: 1.4105465412139893
Iteration: 5/625, Loss: 1.3966829776763916
Iteration: 6/625, Loss: 1.224940538406372
Iteration: 7/625, Loss: 1.2338128089904785
Iteration: 8/625, Loss: 0.981571614742279
Iteration: 9/625, Loss: 1.1209484338760376
Iteration: 10/625, Loss: 1.2982157468795776
Iteration: 11/625, Loss: 1.1213806867599487
Iteration: 12/625, Loss: 1.5706406831741333
Iteration: 13/625, Loss: 1.5134416818618774
Iteration: 14/625, Loss: 1.2641977071762085
Iteration: 15/625, Loss: 1.3383164405822754
Iteration: 16/625, Loss: 1.1944146156311035
Iteration: 17/625, Loss: 1.3878729343414307
Iteration: 18/625, Loss: 1.5067541599273682
Iteration: 19/625, Loss: 1.502952218055725
Iteration: 20/625, Loss: 1.2976605892181396
Iteration: 21/625, Loss: 1.6458079814910889
Iteration: 22/625, Loss: 1.3281151056289673
Iteration: 23/625, Loss: 1.6728981733322144


/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/deeplake/integrations/pytorch/common.py:91: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


Iteration: 1/625, Loss: 1.4857646226882935
Iteration: 2/625, Loss: 1.0035606622695923
Iteration: 3/625, Loss: 1.0273815393447876
Iteration: 4/625, Loss: 1.2415072917938232
Iteration: 5/625, Loss: 1.325743556022644
Iteration: 6/625, Loss: 1.1698577404022217
Iteration: 7/625, Loss: 1.1697138547897339
Iteration: 8/625, Loss: 0.9357360601425171
Iteration: 9/625, Loss: 1.1148006916046143
Iteration: 10/625, Loss: 1.2806280851364136
Iteration: 11/625, Loss: 1.0900505781173706
Iteration: 12/625, Loss: 1.534192442893982
Iteration: 13/625, Loss: 1.4417634010314941
Iteration: 14/625, Loss: 1.2760438919067383
Iteration: 15/625, Loss: 1.277237892150879
Iteration: 16/625, Loss: 1.1486338376998901
Iteration: 17/625, Loss: 1.358311653137207
Iteration: 18/625, Loss: 1.6414333581924438
Iteration: 19/625, Loss: 1.325021743774414
Iteration: 20/625, Loss: 1.2499830722808838
Iteration: 21/625, Loss: 1.5962700843811035
Iteration: 22/625, Loss: 1.2842084169387817
Iteration: 23/625, Loss: 1.661238193511963
Ite

### Saving the model

In [19]:
torch.save(model, "/content/drive/MyDrive/NSP/models/final_model.pth")

In [20]:
saved_model = torch.load("/content/drive/MyDrive/NSP/models/final_model.pth")
saved_model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in